In [18]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
import json

class Onthemarket(scrapy.Spider):
    name = 'onthemarket'
    
    custom_settings = {
        'FEED_FORMAT': 'csv',
        'FEED_URI': 'onthemarket.csv',
        'LOG_FILE': 'onthemarket.log'
    }
    
    headers = {
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
    }
    
    def start_requests(self):
        url = 'https://www.onthemarket.com/for-sale/property/london/'
        
        for page in range(0, 4):
            next_page = url + '?page=' + str(page)
            yield scrapy.Request(url=next_page, headers=self.headers, callback=self.parse)
    
    def parse(self, response):
        # extract data
        for card in response.css('li.property-result'):
            items = {
            	'title': card.css('span.title').css('a::text').get(),
                'address': card.css('span.address').css('a::text').get(),
                'description': card.css('p.description::text').get(),
                'price': card.css('p.price-text').css('a::text').get().encode('ascii', 'ignore').decode('utf-8').strip(),
                'agency': card.css('p.marketed-by').css('a::text').get(),
                'phone': card.css('span.marketed-by-contact').css('strong::text').get(),
                'image_url': card.css('picture').css('img::attr(src)').get()
            }
            
            yield items
    
# run spider
import sys    
if "twisted.internet.reactor" in sys.modules: 
    del sys.modules["twisted.internet.reactor"]

process = CrawlerProcess()
process.crawl(Onthemarket)
process.start()

2022-05-05 22:48:18 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-05-05 22:48:18 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.2.0, Python 3.9.7 (default, Sep 16 2021, 08:50:36) - [Clang 10.0.0 ], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform macOS-10.15.7-x86_64-i386-64bit
2022-05-05 22:48:18 [scrapy.crawler] INFO: Overridden settings:
{'LOG_FILE': 'onthemarket.log'}


In [19]:
import pandas as pd
data = pd.read_csv('onthemarket.csv')

EmptyDataError: No columns to parse from file

In [ ]:
data.head()